## Load dependencies and load data


In [1]:
#Loading dependencies
import requests
from pandas.io.json import json_normalize
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash.dependencies import Input, Output


#Geo map of Cases per country globally
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

app =  dash.Dash()
server =  app.server
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
    
#Reading in data
url = 'https://api.covid19api.com/summary'
r = requests.get(url)
dictr = r.json()
header = dictr['Countries']
df = json_normalize(header)
df = df.drop(df.index[[0,224]])
df = df.drop(columns = 'Slug')
df = df.sort_values(by=['TotalConfirmed'], ascending=False)
df.head()

/home/tshepo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



,Country,CountryCode,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
233,United States of America,US,61483,337065,2532,9619,17448,17448,2020-04-06T16:14:02Z
205,Spain,ES,5478,131646,694,12641,3861,38080,2020-04-06T16:14:02Z
106,Italy,IT,4316,128948,525,15887,819,21815,2020-04-06T16:14:02Z
79,Germany,DE,4031,100123,140,1584,2300,28700,2020-04-06T16:14:02Z
72,France,FR,2925,93773,519,8093,777,16349,2020-04-06T16:14:02Z


In [2]:
#Summing per column
tot_cases = df.TotalConfirmed.sum()
nw_cases = df.NewConfirmed.sum()
tot_deaths = df.TotalDeaths.sum()
new_deaths = df.NewDeaths.sum()
tot_recover = df.TotalRecovered.sum()
new_recover = df.NewRecovered.sum()

#Separating columns

#Corona  cases: Total and New
df_tot_cases = df.loc[:,['Country','TotalConfirmed']]
df_new_cases = df.loc[:,['Country','NewConfirmed']]

#Death Cases: Total and New 
df_tot_death = df.loc[:,['Country','TotalDeaths']]
df_new_death = df.loc[:,['Country','NewDeaths']]

#Recovery cases: Total and new
df_tot_recover = df.loc[:,['Country','TotalRecovered']]
df_new_recover = df.loc[:,['Country','NewRecovered']]
                      
#Sorting cases by total                     
df_cases = df_tot_cases.merge(df_new_cases, on='Country').sort_values(by=['TotalConfirmed'],
                                                                      ascending=False)
df_death = df_tot_death.merge(df_new_death, on='Country').sort_values(by=['TotalDeaths'],
                                                                      ascending=False)
df_recover = df_tot_recover.merge(df_new_recover, on='Country').sort_values(by=['TotalRecovered'], 
                                                                            ascending=False)
# Print statement to be turned to buttons
print(color.BOLD + 'Glabal Covid 19 Virus statistics as at: '
      + color.END,df.Date[1])
print(color.BOLD + color.UNDERLINE +'Total Confirmed Cases: ' 
      + color.END ,'\n','Number of cases', tot_cases)
print(color.BOLD + color.UNDERLINE +'Recoveries World wide: ' 
      + color.END ,'\n','Number of cases', tot_recover)
print(color.BOLD + color.UNDERLINE +'Total Deaths: ' 
      + color.END ,'\n','Number of cases', tot_deaths)

Glabal Covid 19 Virus statistics as at:  2020-04-06T16:14:02Z
Total Confirmed Cases:  
 Number of cases 1225352
Recoveries World wide:  
 Number of cases 258005
Total Deaths:  
 Number of cases 66998


In [3]:
#Geo map of Cases per country globally
data = [ dict(
 type='choropleth',
 locations = df['Country'],
 autocolorscale = True,
 z = df['TotalConfirmed'],
 locationmode = 'country names',
    
 marker = dict(
 line = dict (
 color = 'rgb(255,255,255)',
 width = 2,
 )  ),
 colorbar = dict(
 title = 'Number of cases'
 )
 ) ]
layout = dict(
 autosize=True,
)
fig = go.Figure(data = data, layout = layout)

In [ ]:
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
#Rename Columns for readability
df = df.rename(columns={'NewConfirmed':'New Cases','TotalConfirmed':'Confirmed Cases',
                        'NewDeaths':'New Deaths', 'Total Deaths':'Confirmed Deaths',
                        'NewRecovered':'New Recoveries','TotalRecovered':'Confirmed Recoveries'})

#App
app = dash.Dash(__name__)

app.layout =  html.Div(children=[
    html.H1(
        children='Confirmed Cases per Country',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }
    ),
dcc.Graph(
        id='figure',
        figure=
          go.Figure(data = data, layout = layout)

),
    html.Div(
    [
       html.H1(
        children='Latest Covid 19 Update',
        style={
            'textAlign': 'center',
            'color': 'maroon'
        }
        ),
        dash_table.DataTable(
            id='table',
        # data import
            data=df.to_dict("rows"),
            columns=[{"name": i, "id": i} for i in df.columns],
        # table interactivity
            editable=True,
            sort_action="native",
            sort_mode="multi",
            row_selectable="multi",
            # style table
            style_table={
                'maxHeight': '50ex',
                'overflowY': 'scroll',
                'width': '100%',
                'minWidth': '100%',
            },
            # style cell
            style_cell={
                'fontFamily': 'Open Sans',
                'textAlign': 'center',
                'height': '30px',
                'padding': '10px 22px',
                'whiteSpace': 'inherit',
                'overflow': 'hidden',
                'textOverflow': 'ellipsis',
            },
            style_cell_conditional=[
                {
                    'if': {'column_id': 'State'},
                    'textAlign': 'left'
                },
            ],
            # style header
            style_header={
                'fontWeight': 'bold',
                'backgroundColor': "#3D9970",
                'fontSize':'16px'
            },
            # style filter
            # style data
            style_data_conditional=[
                {
                    # stripped rows
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                },
                {
                    # highlight one row
                    'if': {'row_index': 44},
                    "backgroundColor": "#3D9970",
                    'color': 'white'
                }
            ],
        ),
   ]),
])



if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Apr/2020 18:16:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2020 18:16:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2020 18:16:17] "GET /_dash-layout HTTP/1.1" 200 -
